## Install necessary packages

In [1]:
!pip install transformers datasets 

In [2]:
!pip install rouge.score nltk py7zr

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 774.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.2/379.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.6/138.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 24.4 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=4d859715c58fc0608012f6f7a44a4dd8f152ab8d3299e14791e5334c4441c6bb
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge.score


# Fine tune with transformers

In [3]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Loading dataset error

Sometimes when loading the dataset wile in GPU enviorment it will give the error that it cannot find the *samsum* dataset. The workaround is to load the dataset while in CPU mode then save it localy or on you drive. After that just switch back to GPU and load the dataset from the local file using *load_from_disk()*

In [4]:
data = load_dataset('multi_news')
#data.save_to_disk('/content/samsum')
#data = load_from_disk("/content/drive/MyDrive/samsum")
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-xsum'

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2e145a8e21361ba4ee46fef70640ab946a3e8d425002f104d2cda99a9efca376. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

## Data tokenization

**max_input** and **max_target** can variy depending on the available computing power

In [5]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [6]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['document']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [7]:
tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['document', 'summary'])

  0%|          | 0/45 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

## If memory problems

- sample data to smaller sizes

In [8]:
#sample the data
train_sample = tokenize_data['train'].shuffle(seed=123).select(range(1000))
validation_sample = tokenize_data['validation'].shuffle(seed=123).select(range(500))
test_sample = tokenize_data['test'].shuffle(seed=123).select(range(200))

In [9]:
tokenize_data['train'] = train_sample
tokenize_data['validation'] = validation_sample
tokenize_data['test'] = test_sample

In [10]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

## Training process

In [11]:
#load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Depending on computing power, batch size can go as low as 1 if necessary

In [12]:
batch_size = 1

In [13]:
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
#####################
# metrics
# compute rouge for evaluation 
#####################

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [15]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True

In [16]:
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

Using cuda_amp half precision backend


In [17]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 5000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.543800,2.369417,33.615200,12.164400,20.592400,20.607000,61.966000
2,1.897400,2.393723,34.131300,12.016700,20.645100,20.626400,61.972000
3,1.430100,2.533672,33.408700,11.591400,20.324300,20.292000,62.000000
4,1.059600,2.789590,32.868600,11.105000,19.953800,19.938600,62.000000
5,0.778700,3.033906,33.252000,11.292800,20.106200,20.082900,62.000000
6,0.559800,3.272082,32.898200,10.932900,19.651400,19.623700,61.982000
7,0.407300,3.484862,32.860500,10.553300,19.498800,19.498900,61.992000
8,0.303200,3.679194,33.067300,10.946800,19.750300,19.746000,62.000000
9,0.235900,3.756907,32.722900,10.674900,19.551100,19.564700,61.972000
10,0.193700,3.820121,32.960200,10.777500,19.825300,19.824200,62.000000


Saving model checkpoint to conversation-summ/checkpoint-500
Configuration saved in conversation-summ/checkpoint-500/config.json
Model weights saved in conversation-summ/checkpoint-500/pytorch_model.bin
tokenizer config file saved in conversation-summ/checkpoint-500/tokenizer_config.json
Special tokens file saved in conversation-summ/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
Saving model checkpoint to conversation-summ/checkpoint-1000
Configuration saved in conversation-summ/checkpoint-1000/config.json
Model weights saved in conversation-summ/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in conversation-summ/checkpoint-1000/tokenizer_config.json
Special tokens file saved in conversation-summ/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 500
  Batch size = 1
Saving model checkpoint to conversation-summ/checkpoint-1500
Configuration saved in conversation-summ/checkpo

TrainOutput(global_step=5000, training_loss=0.9409502334594727, metrics={'train_runtime': 7705.5192, 'train_samples_per_second': 1.298, 'train_steps_per_second': 0.649, 'total_flos': 1.083552301056e+16, 'train_loss': 0.9409502334594727, 'epoch': 10.0})

## Testing the fine tuned model

In [18]:
conversation = """
Rann: Hey Harry, how have you been? Long time no see!
Harry: Hey! What a surprise! 
Harry: Yes, you are right, we haven’t seen each other in a long time. How have you been?
Rann: There is an important campaign next week which is keeping me busy otherwise rest is going good in my life. 
Rann: How about you?
Harry: Oh! I just finished a meeting with a very important client of mine and now I finally have some free time. I feel relieved that I’m done with it.
Rann: Good for you then. Hey! Let’s make a plan and catch up with each other after next week. 
Rann: What do you say?
Harry: Sure, why not? Give me a call when you are done with your project.
Rann: Sure, then. 
Rann: Bye, take care.
Harry: Bye buddy.
"""

In [19]:
model_inputs = tokenizer(conversation,  max_length=max_input, padding='max_length', truncation=True)

In [20]:
model_inputs

{'input_ids': [0, 50118, 500, 2279, 35, 11468, 3268, 6, 141, 33, 47, 57, 116, 2597, 86, 117, 192, 328, 50118, 29345, 35, 11468, 328, 653, 10, 2755, 328, 1437, 50118, 29345, 35, 3216, 6, 47, 32, 235, 6, 52, 2220, 17, 27, 90, 450, 349, 97, 11, 10, 251, 86, 4, 1336, 33, 47, 57, 116, 50118, 500, 2279, 35, 345, 16, 41, 505, 637, 220, 186, 61, 16, 2396, 162, 3610, 3680, 1079, 16, 164, 205, 11, 127, 301, 4, 1437, 50118, 500, 2279, 35, 1336, 59, 47, 116, 50118, 29345, 35, 5534, 328, 38, 95, 1550, 10, 529, 19, 10, 182, 505, 3653, 9, 4318, 8, 122, 38, 1747, 33, 103, 481, 86, 4, 38, 619, 15126, 14, 38, 17, 27, 119, 626, 19, 24, 4, 50118, 500, 2279, 35, 2497, 13, 47, 172, 4, 11468, 328, 2780, 17, 27, 29, 146, 10, 563, 8, 2916, 62, 19, 349, 97, 71, 220, 186, 4, 1437, 50118, 500, 2279, 35, 653, 109, 47, 224, 116, 50118, 29345, 35, 9136, 6, 596, 45, 116, 12192, 162, 10, 486, 77, 47, 32, 626, 19, 110, 695, 4, 50118, 500, 2279, 35, 9136, 6, 172, 4, 1437, 50118, 500, 2279, 35, 36255, 6, 185, 575, 4, 501

In [21]:
raw_pred, _, _ = trainer.predict([model_inputs])

***** Running Prediction *****
  Num examples = 1
  Batch size = 1


In [22]:
raw_pred

array([[    2,     0,     0,     0,  2383,   572,    10,   251,  5171,
            9,    55,    87,    10,   353,     6,    41,   470,  9716,
        17065,    11,     5,  2367,   953,  2312,     7,  1349,   159,
           39,   793, 23279,    11,   188,  3324,     8,  2755,   123,
           19,    10,  1028,   486,     4,  2595,     5, 12400,     6,
            5,  9716,     6,  2006,   129,    25,   248,  2279,     6,
         7173,     5,  1028,    71,  3357,   184,    71,     2]])

In [23]:
tokenizer.decode(raw_pred[0])

'</s><s><s><s>– After a long absence of more than a month, an American soldier stationed in the Middle East managed to track down his old buddy in New Zealand and surprise him with a phone call. Per the Telegraph, the soldier, identified only as Rann, answered the phone after returning home after</s>'

In [24]:
tokenizer.decode(raw_pred[0])

'</s><s><s><s>– After a long absence of more than a month, an American soldier stationed in the Middle East managed to track down his old buddy in New Zealand and surprise him with a phone call. Per the Telegraph, the soldier, identified only as Rann, answered the phone after returning home after</s>'